In [ ]:
import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.models import ExpertRating, Video, VideoRating
import numpy as np
from backend.rating_fields import VIDEO_FIELDS
from tqdm import tqdm
import pandas as pd

In [ ]:
def get_mismatches():
    """Get all mismatches dataframe."""
    ratings = ExpertRating.objects.all()
    mismatches = []

    for r in tqdm(ratings):
        user = r.user

        v1, v2 = r.video_1, r.video_2
        vr1 = VideoRating.objects.get(user=user, video=v1)
        vr2 = VideoRating.objects.get(user=user, video=v2)

        for f in VIDEO_FIELDS:
            first_vs_second_rating = getattr(r, f)
            first_vs_second_model  = getattr(vr2, f) - getattr(vr1, f)

            mismatch = {
                'user': user,
                'rating': r,
                'feature': f,
                'input_comparison': first_vs_second_rating,
                'rating_1_model': getattr(vr1, f),
                'rating_2_model': getattr(vr2, f),
                'input_comparison_signed': first_vs_second_rating - 50,
                'rating_delta_21': getattr(vr2, f) - getattr(vr1, f),
            }

            mismatch['mismatch'] = np.sign(mismatch['input_comparison_signed']) != np.sign(mismatch['rating_delta_21'])

            mismatches.append(mismatch)
    mismatches = pd.DataFrame(mismatches)
    return mismatches

In [ ]:
mismatches = get_mismatches()

In [ ]:
np.mean(mismatches.mismatch)

In [ ]:
mismatches[mismatches.mismatch]